In [30]:
from mesa import Agent, Model
from mesa.time import StagedActivation
import random
import numpy as np

random.seed(42)

In [41]:
## Parameters 
N = 5
prop_adversaries = .3
coin_toss = .8

In [42]:
class DummyAgent(Agent):
    def __init__(self, id, model):

        ## Apparently this is always needed
        super().__init__(id, model)

        ## Every agent has access to the model, bad design, I know, but I am lazy :( 
        self.model_state = model
        ## Unique ID, based on integer
        self.id = id

        ## This is the list of "others" ID to choose from
        self.others = list(range(0,N))
        self.others.remove(self.id)

        ## The belief vector that corresponds to the the IDs
        ## It is the belief that an agent is good
        self.belief = [1 - prop_adversaries] *  (N - 1)

        ## The belief vector that corresponds to the the IDs
        self.votes = []

    ## Should be fixed for every agent
    def select_from_belief(self):

        ## For conversion to a valid probability distribution
        normalizing_constant = sum(self.belief)
        
        ## If the agent does not trust anyone, then
        if normalizing_constant == 0:
            ## No one was chosen
            self.model_state.chose = False
            return -1
        
        else:

            selection_probability = [b / normalizing_constant for b in self.belief]
            # Using random.choices() for a single selection
            selected_item = random.choices(self.others, weights=selection_probability, k=1)[0]

            return selected_item
    
    def choose(self):

        # Implementation of choose action
        if self.model_state.whose_turn == self.id:

            choice = self.select_from_belief()

            print("I am " + str(self.id))
            print("and I choose " + str(choice))

            self.model_state.chosen.append(choice)
            
        else:

            pass
    
    def vote(self):

        # Implementation of interaction with other agents
        # print("interact")

        if self.model_state.chose:

            chooser = self.model_state.whose_turn
            chosen = self.model_state.chosen[-1]

            belief_chooser, belief_chosen = None, None

            if chooser == self.id:
                belief_chooser = 1
            elif chosen == self.id:
                belief_chosen = 1
            
            if belief_chooser is None:
                belief_chooser = self.belief[self.others.index(chooser)]

            if belief_chosen is None:
                belief_chosen = self.belief[self.others.index(chosen)]

            if belief_chooser + belief_chosen > 1:
                print("Approved")
                self.votes.append(1)
            else:
                print("Nah")
                self.votes.append(0)

        pass
    
    def act(self):
        # Update internal state
        #print("update")
        pass

    def update(self):
        pass


class MyModel(Model):
    def __init__(self, N=N):
        
        super().__init__()

        self.num_agents = N

        self.schedule = StagedActivation(self, stage_list=["choose", "vote", "act", "update"])
        self.round = 0 
        self.whose_turn = self.round % N

        self.chose = True
        self.chosen = []
        
        # Change this up a little
        for i in range(self.num_agents):  # Example: creating 10 agents
            agent = DummyAgent(i, self)
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()
        self.round += 1
        self.whose_turn = self.round % N
        self.chose = True


In [43]:
model = MyModel(N)

for i in range(10):
    model.step()


I am 0
and I choose 1
0.7
Approved
0.7
Approved
0.7
0.7
Approved
0.7
0.7
Approved
0.7
0.7
Approved
I am 1
and I choose 4
0.7
0.7
Approved
0.7
Approved
0.7
0.7
Approved
0.7
0.7
Approved
0.7
Approved
I am 2
and I choose 3
0.7
0.7
Approved
0.7
0.7
Approved
0.7
Approved
0.7
Approved
0.7
0.7
Approved
I am 3
and I choose 4
0.7
0.7
Approved
0.7
0.7
Approved
0.7
0.7
Approved
0.7
Approved
0.7
Approved
I am 4
and I choose 2
0.7
0.7
Approved
0.7
0.7
Approved
0.7
Approved
0.7
0.7
Approved
0.7
Approved
I am 0
and I choose 3
0.7
Approved
0.7
0.7
Approved
0.7
0.7
Approved
0.7
Approved
0.7
0.7
Approved
I am 1
and I choose 4
0.7
0.7
Approved
0.7
Approved
0.7
0.7
Approved
0.7
0.7
Approved
0.7
Approved
I am 2
and I choose 1
0.7
0.7
Approved
0.7
Approved
0.7
Approved
0.7
0.7
Approved
0.7
0.7
Approved
I am 3
and I choose 2
0.7
0.7
Approved
0.7
0.7
Approved
0.7
Approved
0.7
Approved
0.7
0.7
Approved
I am 4
and I choose 3
0.7
0.7
Approved
0.7
0.7
Approved
0.7
0.7
Approved
0.7
Approved
0.7
Approved


In [34]:
model.chosen

[1, 2, 0, 2, 2, 4, 0, 1, 0, 0]

In [11]:
# Example lists
items = ['apple', 'banana', 'cherry']
probabilities = [0.1, 0.6, 0.3]

# Using random.choices() for a single selection
selected_item = random.choices(items, weights=probabilities, k=1)[0]
selected_item

'apple'

In [6]:
selected_item

'cherry'

In [18]:
[0.1, 0.6, 0.3] / sum([0.1, 0.6, 0.3])

TypeError: unsupported operand type(s) for /: 'list' and 'float'